In [18]:
# ADB初始化

devices = ['','FA8251A00744','FA7981A10296','HT7A61A00452','HT8571A02153','FA7AR1A06690','FA81S1A02788','FA81T1A00545','FA7BS1A01242','FA7AR1A00090','HT8191A02068','FA8281A00672','FA8191A02532','HT7C61A04994','HT8191A00307','HT83M1A04115','FA79C1A03546','HT8221A01014','HT7BA1A00045','HT83G1A00014','FA7B81A02905','HT81A1A00339','FA81V1A00444','HT7B11A01104','HT8611A04012','HT7C61A06443','HT7B61A04709','FA7B11A05410','FA7AY1A03300','HT82C1A02048','HT81A1A02031']
os.system('adb kill-server')
os.system('adb start-server')
os.system('adb devices')

* daemon not running; starting now at tcp:5037


List of devices attached
FA7981A10296	device
FA79C1A03546	device
FA7AR1A00090	device
FA7AR1A06690	device
FA7AY1A03300	device
FA7B11A05410	device
FA7B81A02905	device
FA7BS1A01242	device
FA8191A02532	device
FA81S1A02788	device
FA81T1A00545	device
FA81V1A00444	device
FA8251A00744	device
FA8281A00672	device
HT7A61A00452	device
HT7B11A01104	device
HT7B61A04709	device
HT7C61A04994	device
HT7C61A06443	device
HT8191A00307	device
HT8191A02068	device
HT81A1A00339	device
HT8221A01014	device
HT82C1A02048	device
HT83G1A00014	device
HT83M1A04115	device
HT8571A02153	device
HT8611A04012	device
RF8M20DCVVN	unauthorized



* daemon started successfully


0

In [142]:
# Python初始化

import os
import re
import subprocess
import time
from multiprocessing import Pool
from queue import Queue

def getResult(str, dev, isPrint = 0):
    #生成命令
    scmd = 'adb -s '+ dev + ' ' + str
    if isPrint == 1:
        print(scmd)

    #获取结果
    result = os.popen(scmd)
    stdout = result.read()
    return stdout

def checkConnection(dev, recTime = 10):
    command = 'adb -s '+ dev + ' ' + 'shell ping -c 2 -W 2 www.google.com'  # 具体命令

    # 开始执行adb命令
    p_obj = subprocess.Popen(
            args=command,
            stdin=None, stdout=subprocess.PIPE,
            stderr=subprocess.PIPE, shell=True)
    
    time.sleep(recTime)
    
    flag = p_obj.poll()
    p_obj.kill()

    if flag == 0:
        return 1
    else:
        return 0

def connectVPN(dev, recTime = 10):
    # 先杀死VPN和TT
    getResult('shell am force-stop com.surfshark.vpnclient.android', dev)
    getResult('shell am force-stop com.zhiliaoapp.musically', dev)

    # 设置代理
    getResult('shell settings put global http_proxy 192.168.1.188:7890', dev)

    # 打开VPN
    getResult('shell am start com.surfshark.vpnclient.android/.StartActivity', dev)
    time.sleep(2)

    # 连接VPN
    getResult('shell input tap 535 1547', dev)

    time.sleep(1)

    # 尝试关掉提醒对话框
    getResult('shell input tap 788 993', dev)

    time.sleep(recTime-1)
   
    getResult('shell settings put global http_proxy :0', dev)

def doAction(str, dev, delayTime = 0):
    _command = 'adb -s '+ dev + ' ' + str
    time.sleep(delayTime)
    subprocess.Popen(_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf-8")

In [182]:
#进行adb devices的全部设备连接检查

result = os.popen('adb devices')
stdout = result.read()

flag = 0

for i in range(1,31):
    if devices[i] not in stdout:
        print(i, "no found")
        flag = 1

if flag == 0:
    print("all good")


18 no found
30 no found


In [144]:
# 批量开关机并返回主页面

isOpen = 1  # 1为开机，0为关机

devicesPool = devices[1:31]

for dev in devicesPool:
    stdout = getResult('shell dumpsys window policy', dev)

    if isOpen == 1:
        if 'interactiveState=INTERACTIVE_STATE_SLEEP' in stdout:
            doAction('shell input keyevent 26', dev)
            
        if 'showing=true' in stdout:
            doAction('shell input keyevent 82', dev, 0.25)

    else:
        if 'interactiveState=INTERACTIVE_STATE_SLEEP' not in stdout:
            doAction('shell input keyevent 26', dev)

if isOpen == 1:
    for dev in devicesPool:
        doAction('shell input keyevent 3', dev)



adb: device 'HT7BA1A00045' not found
adb: device 'HT81A1A02031' not found


In [181]:
#%%writefile test.py
# 并发检测网络并打开VPN

import os
import re
import subprocess
import time
from multiprocessing import Pool
from queue import Queue
import test

devices = ['','FA8251A00744','FA7981A10296','HT7A61A00452','HT8571A02153','FA7AR1A06690','FA81S1A02788','FA81T1A00545','FA7BS1A01242','FA7AR1A00090','HT8191A02068','FA8281A00672','FA8191A02532','HT7C61A04994','HT8191A00307','HT83M1A04115','FA79C1A03546','HT8221A01014','HT7BA1A00045','HT83G1A00014','FA7B81A02905','HT81A1A00339','FA81V1A00444','HT7B11A01104','HT8611A04012','HT7C61A06443','HT7B61A04709','FA7B11A05410','FA7AY1A03300','HT82C1A02048','HT81A1A02031']

def getResult(str, dev, isPrint = 0):
    #生成命令
    scmd = 'adb -s '+ dev + ' ' + str
    if isPrint == 1:
        print(scmd)

    #获取结果
    result = os.popen(scmd)
    stdout = result.read()
    return stdout

def checkConnection(dev, recTime = 10):
    command = 'adb -s '+ dev + ' ' + 'shell ping -c 2 -W 2 www.google.com'  # 具体命令

    # 开始执行adb命令
    p_obj = subprocess.Popen(
            args=command,
            stdin=None, stdout=subprocess.PIPE,
            stderr=subprocess.PIPE, shell=True)

    time.sleep(0.1)

    if 1 == p_obj.poll():
        return -1

    time.sleep(recTime)
    
    flag = p_obj.poll()

    p_obj.kill()
    if flag == 0:
        return 1
    else:
        return 0

def connectVPN(dev, recTime = 10, isRepeat = 0):
    # 先杀死VPN和TT
    getResult('shell am force-stop com.surfshark.vpnclient.android', dev)
    getResult('shell am force-stop com.zhiliaoapp.musically', dev)

    # 设置代理
    getResult('shell settings put global http_proxy 192.168.1.188:7890', dev)

    # 打开VPN
    getResult('shell am start com.surfshark.vpnclient.android/.StartActivity', dev)
    time.sleep(2)

    # 连接VPN
    getResult('shell input tap 535 1547', dev)

    time.sleep(1)

    # 尝试关掉提醒对话框
    getResult('shell input tap 788 993', dev)

    if isRepeat == 1:
        getResult('shell input tap 535 1547', dev)
        time.sleep(0.5)
        getResult('shell input tap 535 1147', dev)
        time.sleep(0.5)
        getResult('shell input tap 535 1547', dev)
        time.sleep(1)

    time.sleep(recTime-1)
   
    getResult('shell settings put global http_proxy :0', dev)

def doVPN(dev):
    print("开始检查设备" + str(devices.index(dev)))
    
    tryTimes = 1
    flag = 0

    while(True):
        flag = checkConnection(dev, tryTimes * 5 + 6)
        if (flag != 0) or (tryTimes >= 4):
            break
        # 设备没有联网，立刻重连
        print("设备{}无网络连接，第{}次重连中... ".format(devices.index(dev), tryTimes))
        connectVPN(dev, tryTimes * 5 + 6, (tryTimes >= 2))
        tryTimes = tryTimes + 1
    
    if flag == -1:
        print("设备{} ADB未成功连接".format(devices.index(dev)))
    elif tryTimes <= 3:
        # 设备正常联网
        print("设备{}网络正常，打开SurfShark页面".format(devices.index(dev)))
        getResult('shell am start com.surfshark.vpnclient.android/.StartActivity', dev)
    else:
        # 设备未正常联网
        print("设备{}未正常联网，返回主页面".format(devices.index(dev)))
        getResult('shell input keyevent 26', dev)

if __name__ == '__main__':
    pool = Pool()
    res = pool.map(test.doVPN, devices[1:31])


开始检查设备1
开始检查设备2
开始检查设备3
开始检查设备4
开始检查设备5
开始检查设备7
开始检查设备6
开始检查设备8
开始检查设备9
开始检查设备10
设备1网络正常，打开SurfShark页面
设备6网络正常，打开SurfShark页面
设备2网络正常，打开SurfShark页面
设备4网络正常，打开SurfShark页面
设备3网络正常，打开SurfShark页面
设备8网络正常，打开SurfShark页面
设备5网络正常，打开SurfShark页面
设备7网络正常，打开SurfShark页面
设备9无网络连接，第1次重连中... 
设备10无网络连接，第1次重连中... 
开始检查设备11
开始检查设备12
开始检查设备13
开始检查设备14
开始检查设备15
开始检查设备16
开始检查设备17
开始检查设备18
设备18 ADB未成功连接
开始检查设备19
设备11网络正常，打开SurfShark页面
设备12网络正常，打开SurfShark页面
设备13网络正常，打开SurfShark页面
设备14无网络连接，第1次重连中... 
设备17网络正常，打开SurfShark页面
设备15无网络连接，第1次重连中... 
设备16无网络连接，第1次重连中... 
开始检查设备20
开始检查设备21
开始检查设备22
开始检查设备23
设备19网络正常，打开SurfShark页面
开始检查设备24
设备20网络正常，打开SurfShark页面
设备21网络正常，打开SurfShark页面
设备22网络正常，打开SurfShark页面
设备23网络正常，打开SurfShark页面
开始检查设备25
开始检查设备26
开始检查设备27
开始检查设备28
设备24网络正常，打开SurfShark页面
开始检查设备29
设备9网络正常，打开SurfShark页面
开始检查设备30
设备10网络正常，打开SurfShark页面
设备30 ADB未成功连接
设备25无网络连接，第1次重连中... 
设备26网络正常，打开SurfShark页面
设备27无网络连接，第1次重连中... 
设备28无网络连接，第1次重连中... 
设备29网络正常，打开SurfShark页面
设备15网络正常，打开SurfShark页面
设备16网络正常，打开SurfShark页面
设

In [136]:
# 批量打开应用

for dev in devices[1:31]:
    # 打开VPN
    #getResult('shell am start com.surfshark.vpnclient.android/.StartActivity', dev)

    # 打开智云控
    doAction('shell am start org.zhiyunkong.zhiyunkong/.ui.splash.SplashActivity', dev)

    # 打开VPN
    #getResult('shell am start com.surfshark.vpnclient.android/.StartActivity', dev)



In [137]:
# 批量返回主页面

for dev in devices[1:31]:
   doAction('shell input keyevent 3', dev)

In [139]:
# 设置代理

for dev in devices[1:31]:
   # 开启
   #getResult('shell settings put global http_proxy 192.168.1.188:7890', dev)

   # 关闭
   doAction('shell settings put global http_proxy :0', dev)

adb: device 'HT7BA1A00045' not found
adb: device 'HT81A1A02031' not found


In [105]:
# 网络——仅检查网络连接

for dev in devices[19:21]:
   print("正在检查设备" + str(devices.index(dev)) + ":", end="")
   
   stdout = getResult('shell ping -c 3 -W 2 www.google.com', dev)

   if 'received' not in stdout:
      print(" 没有网络连接")
   else:
      print(" 有网络连接")


正在检查设备19: 有网络连接
正在检查设备20: 有网络连接


In [114]:
# 安装应用

for dev in devices[2:9]:
   doAction('install /Users/calvert/Documents/Rockflow/TikTok/脚本/zhiyunkong112.apk', dev)

In [ ]:
# 监听设备的Activity。需要利用Terminal

for dev in devices[9:10]:
    print('adb -s '+ dev + ' ' + 'logcat | grep ActivityTaskManager')